## Evaluation of fine-tuned BERT base uncased models on validation datasets
### this notebook contains functions and fragments of code taken from the Jypyter notebook available on HuggingFace GitHub https://github.com/huggingface/notebooks/blob/main/examples/question_answering.ipynb

In [9]:
import transformers
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset, load_metric
import pandas as pd
from datasets import ClassLabel, Sequence
from transformers import AutoTokenizer
from transformers import EarlyStoppingCallback
from transformers import default_data_collator
from datasets import Dataset
import collections
import numpy as np
from tqdm.auto import tqdm

In [10]:
# parameter for metrics
squad_v2 = False

# parameter for model
model_checkpoint = "bert-base-uncased"

# parameter for batch size
batch_size = 16

In [11]:
# name for the model
m_name = 'squad_base'

# path to the saved BERT model
saved_model = 'models/saved_baseline_model'

In [12]:
# loading of tokenizer from saved model
tokenizer = AutoTokenizer.from_pretrained(saved_model)
# loading of saved fine-tuned model
model = AutoModelForQuestionAnswering.from_pretrained(saved_model)
# data collator
data_collator = default_data_collator
#loading of metrics for evaluation
metric = load_metric("squad_v2" if squad_v2 else "squad")

Didn't find file models/saved_baseline_model/added_tokens.json. We won't load it.
loading file models/saved_baseline_model/vocab.txt
loading file models/saved_baseline_model/tokenizer.json
loading file None
loading file models/saved_baseline_model/special_tokens_map.json
loading file models/saved_baseline_model/tokenizer_config.json
loading configuration file models/saved_baseline_model/config.json
Model config BertConfig {
  "_name_or_path": "models/saved_baseline_model",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float3

In [13]:
# parameters from the QA HuggingFace Jupyter Notebook
pad_on_right = tokenizer.padding_side == "right"
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [14]:
# training arguments for the saved model
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad_with_callbacks",
    evaluation_strategy = "steps",
    eval_steps = 200,
    save_steps = 200,
    logging_steps = 200,
    save_total_limit = 5,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",
    load_best_model_at_end=True,
)

PyTorch: setting up devices


In [15]:
# Trainer for model evaluation
trainer = Trainer(
    model,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,    
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

In [16]:
# Preprocessing function for train dataset from the HuggingFace Jupyter notebook
# with original comments
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [17]:
# Preprocessing function for validation dataset from the HuggingFace Jupyter notebook
# with original comments
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [18]:
# Postprocessing function from the HuggingFace Jupyter notebook
# with original comments
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    if len(offset_mapping[start_index]) == 0 or len(offset_mapping[end_index]) == 0: 
                        continue
                                        
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [22]:
def model_evaluation_on_dataset(dataset, save_dataframe_with_predictions = False, name = 'model_name'):
    """Model evaluation on specific dataset
    Calls previous functions and evaluate the dataset on the model for exact match and F1

    Args:
        dataset (Dataset): validation dataset
        save_dataframe_with_predictions (bool, optional): flag for saving the dataset with prediction. Defaults to False.
        name (str, optional): name of the model. Defaults to 'model_name'.

    Returns:
        _type_: _description_
    """
    validation_features = dataset.map(
        prepare_validation_features,
        batched=True,
        remove_columns=dataset.column_names
    )

    raw_predictions = trainer.predict(validation_features)

    print(raw_predictions)
    
    validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

    final_predictions = postprocess_qa_predictions(dataset, validation_features, raw_predictions.predictions)

    predictions = [v for k, v in final_predictions.items()]
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in dataset]
    
    if save_dataframe_with_predictions:
        data = pd.DataFrame(dataset)   
        data['prediction_text'] = predictions
        data.to_json('./datasets/' + name + '.json', orient='records')

    return metric.compute(predictions=formatted_predictions, references=references)

### Evaluation of models on datasets



### SQuAD dataset 

In [20]:
datasets = load_dataset("squad")

Reusing dataset squad (/home/luki/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00,  8.22it/s]


In [23]:
metrics = model_evaluation_on_dataset(datasets['validation'], save_dataframe_with_predictions = True, name = m_name)
metrics

Loading cached processed dataset at /home/luki/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-c004983476855886.arrow
The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10784
  Batch size = 16


PredictionOutput(predictions=(array([[-6.1160293, -7.85931  , -8.013237 , ..., -8.699536 , -8.705282 ,
        -8.710753 ],
       [-6.307056 , -7.8310676, -7.994099 , ..., -8.695235 , -8.699649 ,
        -8.704299 ],
       [-6.54725  , -7.251878 , -7.8062453, ..., -8.6584835, -8.682704 ,
        -8.670699 ],
       ...,
       [-4.9577866, -8.093667 , -8.379602 , ..., -8.673    , -8.688061 ,
        -8.692666 ],
       [-6.0060143, -8.09563  , -8.027556 , ..., -8.755645 , -8.742174 ,
        -8.733725 ],
       [-4.7843885, -8.220863 , -8.423353 , ..., -8.705527 , -8.720462 ,
        -8.7262945]], dtype=float32), array([[-5.754394 , -8.28669  , -8.314434 , ..., -8.650862 , -8.645545 ,
        -8.639945 ],
       [-6.019397 , -8.302401 , -8.276548 , ..., -8.654089 , -8.6505785,
        -8.64501  ],
       [-6.366077 , -8.460493 , -8.7973   , ..., -8.630322 , -8.611618 ,
        -8.616931 ],
       ...,
       [-4.4436255, -8.4410305, -8.691105 , ..., -8.70753  , -8.690399 ,
        -8

100%|██████████| 10570/10570 [00:22<00:00, 462.60it/s]


{'exact_match': 80.19867549668874, 'f1': 87.57572428424854}

In [15]:
# logging of the metrics
with open("evaluation_of_models_on_datasets.csv", "a") as file_append:
        file_append.write(f"\nSQuAD,{m_name},{metrics['exact_match']},{metrics['exact_match']-80.19867549668874},{metrics['f1']},{metrics['f1']-87.57572428424854}")

### AdversarialQA dataset

In [16]:
datasets = load_dataset("adversarial_qa", "adversarialQA")

Reusing dataset adversarial_qa (/home/luki/.cache/huggingface/datasets/adversarial_qa/adversarialQA/1.0.0/92356be07b087c5c6a543138757828b8d61ca34de8a87807d40bbc0e6c68f04b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
datasets['validation']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'metadata'],
    num_rows: 3000
})

In [18]:
data = pd.DataFrame(datasets['validation'])

In [19]:
data.to_json('adversarialQA_validation.json', orient='records')

In [20]:
metrics = model_evaluation_on_dataset(datasets['validation'])
metrics

  0%|          | 0/3 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3010
  Batch size = 16


Post-processing 3000 example predictions split into 3010 features.


  0%|          | 0/3000 [00:00<?, ?it/s]

{'exact_match': 18.666666666666668, 'f1': 30.080856274967804}

In [21]:
# logging of the metrics
with open("evaluation_of_models_on_datasets.csv", "a") as file_append:
        file_append.write(f"\nAdversarialQA,{m_name},{metrics['exact_match']},{metrics['exact_match']-19.866666666666667},{metrics['f1']},{metrics['f1']-31.299247172832363}")

### Natural Questions dataset

In [4]:
data = pd.read_json('./datasets/nq_dev_formatted.json')

In [5]:
data

,id,context,question,answers
0,6915606477668963328,"In logical argument and mathematical proof, th...",what do the 3 dots mean in math,{'text': ['the therefore sign (∴) is generally...
1,-5004457603684974592,The Super Bowl 50 Halftime Show took place on ...,who is playing the halftime show at super bowl...,{'text': ['British rock group Coldplay[1] with...
2,8494342737238168576,"Matthew MacKendree ""Matt"" Lanter (born April 1...",star wars the clone wars anakin voice actor,"{'text': ['Matthew MacKendree ""Matt"" Lanter'],..."
3,7217222058435937280,The forum is best known for its annual meeting...,where was the world economic forum held this year,"{'text': ['Davos', 'Davos, a mountain resort i..."
4,5533906981191706624,"Since 1947, there have been eight Chief Minist...",who was the first chief minister of west bengal,"{'text': ['Prafulla Chandra Ghosh', 'Prafulla ..."
...,...,...,...,...
2351,-8747637469328088064,"Queen Elizabeth II is the sovereign, and her h...",who is next in line to inherit the british throne,"{'text': ['Charles, Prince of Wales', 'her eld..."
2352,-3916198822947717632,The memory in flash drives is commonly enginee...,how many writes does a flash drive have,"{'text': ['around 3,000-5,000 program-erase cy..."
2353,-3200873089876979712,Thirty-three amendments to the United States C...,how many ammendments to the constitution have ...,"{'text': ['Thirty-three', 'Twenty-seven'], 'an..."
2354,-4445560055142980608,3H (atomic mass 7000301604928199000♠3.01604928...,what is the mass number of hydrogen isotope th...,"{'text': ['3.01604928199(23) u', '700030160492..."


In [24]:
dataset = Dataset.from_pandas(data)

In [25]:
metrics = model_evaluation_on_dataset(dataset)
metrics

  0%|          | 0/3 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2388
  Batch size = 16


Post-processing 2356 example predictions split into 2388 features.


  0%|          | 0/2356 [00:00<?, ?it/s]

{'exact_match': 58.31918505942275, 'f1': 70.08670117707076}

In [2]:
# logging of the metrics
with open("evaluation_of_models_on_datasets.csv", "a") as file_append:
        file_append.write(f"\nNaturalQuestions,{m_name},{metrics['exact_match']},{metrics['exact_match']-55.730050933786075},{metrics['f1']},{metrics['f1']-67.82058427738443}")

NameError: name 'm_name' is not defined

### TriviaQA dataset

In [6]:
data = pd.read_json('./datasets/triviaqa_dev_formatted.json')

In [7]:
data

,id,context,question,answers
0,tc_49--Kiss_You_All_Over.txt,`` Kiss You All Over '' is a 1978 song perform...,Who had a 70s No 1 hit with Kiss You All Over?,"{'text': ['Exile'], 'answer_start': [62]}"
1,tc_56--Kathleen_Ferrier.txt,"Kathleen Mary Ferrier , CBE ( 22 April 1912 - ...",What claimed the life of singer Kathleen Ferrier?,"{'text': ['Cancer'], 'answer_start': [2488]}"
2,tc_106--Lauren_Bacall.txt,"Lauren Bacall ( , born Betty Joan Perske ; Sep...",Which actress was voted Miss Greenwich Village...,"{'text': ['Bacall'], 'answer_start': [7]}"
3,tc_137--Michael_Jackson.txt,"Michael Joseph Jackson ( August 29 , 1958 – Ju...",What was the name of Michael Jackson's autobio...,"{'text': ['moonwalk'], 'answer_start': [1508]}"
4,tc_217--Tanzania.txt,"Tanzania , This approximates the Kiswahili pro...",Which volcano in Tanzania is the highest mount...,"{'text': ['Kilimanjaro'], 'answer_start': [478]}"
...,...,...,...,...
9830,qg_4604--National_Guard_of_the_United_States.txt,"The National Guard of the United States , part...","With a motto of Always Ready, Always There, wh...","{'text': ['National Guard'], 'answer_start': [4]}"
9831,qg_4615--Whoville.txt,Whoville is a fictional town created by author...,Who tried to steal Christmas from the town of ...,"{'text': ['Grinch'], 'answer_start': [161]}"
9832,qg_4628--Winter_Wonderland.txt,"`` Winter Wonderland '' is a winter song , pop...",What is the name of the parson mentioned in th...,"{'text': ['Brown'], 'answer_start': [1908]}"
9833,qg_4628--Frosty's_Winter_Wonderland.txt,Frosty 's Winter Wonderland is a 1976 animated...,What is the name of the parson mentioned in th...,"{'text': ['Brown'], 'answer_start': [2032]}"


In [29]:
dataset = Dataset.from_pandas(data)

In [30]:
metrics = model_evaluation_on_dataset(dataset)
metrics

  0%|          | 0/10 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 36286
  Batch size = 16


KeyboardInterrupt: 

In [ ]:
# logging of the metrics
with open("evaluation_of_models_on_datasets.csv", "a") as file_append:
        file_append.write(f"\nTriviaQA,{m_name},{metrics['exact_match']},{metrics['exact_match']-37.580071174377224},{metrics['f1']},{metrics['f1']-46.881850685798064}")